# Exception Handling with Middleware

This notebook demonstrates how to use middleware for robust error handling and recovery.

## Key Concepts

- **Error Catching**: Catch exceptions from tool invocations
- **Graceful Degradation**: Provide fallback responses
- **Error Logging**: Track failures for debugging

## Prerequisites

- Azure OpenAI endpoint and deployment configured in `.env`
- `agent-framework` package installed
- Azure CLI authentication

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../.env')

project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME")

print(f"Project Endpoint: {project_endpoint}")
print(f"Model Deployment: {model_deployment_name}")

In [ ]:
from collections.abc import Awaitable, Callable
from typing import Annotated
from agent_framework import AgentMiddleware, FunctionInvocationContext
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field

## Define Unstable Data Service Tool

This simulated data service demonstrates exception handling by throwing a TimeoutError.

In [ ]:
def unstable_data_service(
    query: Annotated[str, Field(description="The data query to execute.")],
) -> str:
    """A simulated data service that sometimes throws exceptions."""
    # Simulate failure
    raise TimeoutError("Data service request timed out")

## Exception Handling Middleware

This middleware demonstrates:
- Catching exceptions thrown by functions
- Providing graceful error responses
- Overriding function results when errors occur
- Implementing centralized exception handling

In [ ]:
async def exception_handling_middleware(
    context: FunctionInvocationContext, next: Callable[[FunctionInvocationContext], Awaitable[None]]
) -> None:
    function_name = context.function.name

    try:
        print(f"[ExceptionHandlingMiddleware] Executing function: {function_name}")
        await next(context)
        print(f"[ExceptionHandlingMiddleware] Function {function_name} completed successfully.")
    except TimeoutError as e:
        print(f"[ExceptionHandlingMiddleware] Caught TimeoutError: {e}")
        # Override function result to provide custom message in response.
        context.result = (
            "Request Timeout: The data service is taking longer than expected to respond.",
            "Respond with message - 'Sorry for the inconvenience, please try again later.'",
        )

## Example: Using Exception Handling Middleware

In [ ]:
async def main():
    """Example demonstrating exception handling with middleware."""
    print("=== Exception Handling Middleware Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="DataAgent",
            instructions="You are a helpful data assistant. Use the data service tool to fetch information for users.",
            tools=unstable_data_service,
            middleware=exception_handling_middleware,
        ) as agent,
    ):
        query = "Get user statistics"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result}")

await main()

## Key Takeaways

- Middleware can catch exceptions and provide graceful error handling
- Use try-except blocks to catch specific error types (like TimeoutError)
- Override `context.result` to provide user-friendly error messages
- Centralized exception handling prevents raw exceptions from reaching end users
- Can implement retry logic, fallback mechanisms, or error reporting

## Next Steps

- **Middleware termination** (notebook 7) for early exit scenarios
- **Result override** (notebook 8) for modifying outcomes